In [1]:
from pprint import pprint
import os
import sys

#name of the cluster
your_cluster_name = "mustafa7"

## The private key pair for accessing cluster.
private_key = "/home/mustafa/interns_oregon_key.pem"

## If delete cfncluster after job is done./
delete_cfncluster= False

print("variables set")

variables set


In [2]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager
from util import AddonsManager

## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

cluster mustafa7 does exist.

Status: CREATE_COMPLETE
Status: CREATE_COMPLETE
MasterServer: RUNNING
MasterServer: RUNNING
Output:"MasterPublicIP"="34.214.180.149"
Output:"MasterPrivateIP"="172.31.33.31"
Output:"GangliaPublicURL"="http://34.214.180.149/ganglia/"
Output:"GangliaPrivateURL"="http://172.31.33.31/ganglia/"

connecting
connected


### Check Scripts on Cluster

This section containss cells that can be used to check which scripts are currently on the cluster being used. 

#### Get all supported pipelines

In [3]:
#This cell must be run before others in this section

scripts = AddonsManager.get_scripts_dict(ssh_client)
print()
print("Supported Pipelines:", AddonsManager.get_all_pipeline_names(scripts))

Executing python /shared/workspace/Pipelines/util/GetScripts.py

Supported Pipelines: ['RNASeq', 'ChiPSeq', 'DNASeq', 'SmallRNASeq']


#### Check which workflows are in a given pipeline

In [4]:
#can be set to a supported pipeline name or "all"
target_pipeline = "all"

print("Supported Workflows in {} Pipeline(s): ".format(target_pipeline), end="")
pprint(AddonsManager.get_workflows_in_pipeline(scripts, target_pipeline), indent=2)

Supported Workflows in all Pipeline(s): 
{ 'ChiPSeq': ['homer'],
  'DNASeq': ['bwa_gatk', 'bwa_mutect'],
  'RNASeq': ['star_rsem', 'star_gatk', 'star_htseq', 'kallisto'],
  'SmallRNASeq': ['bowtie2']}


#### Check which shell scripts are in a given pipeline or workflow

In [5]:
#can be set to a supported pipeline or "all"
target_pipeline = "all"

#can be set to a support workflow or "all"
#if target_pipeline == "all" then this variable is ignored
target_workflow = "all"

pprint(AddonsManager.get_scripts(scripts, target_pipeline, target_workflow))

{'All Pipelines': ['fastqc.sh',
                   'multiqc.sh',
                   'trim.sh',
                   'run.sh',
                   'merge_counts.sh'],
 'ChiPSeq': {'All Workflows': [],
             'homer': ['pos2bed.sh',
                       'find_motifs_genome.sh',
                       'annotate_peaks.sh',
                       'make_tag_directory.sh',
                       'bowtie.sh',
                       'findpeaks.sh',
                       'make_UCSC_file.sh']},
 'DNASeq': {'All Workflows': ['merge.sh',
                              'split.sh',
                              'sort.sh',
                              'bwa.sh',
                              'post.sh',
                              'dedup.sh'],
            'bwa_gatk': ['haplo.sh', 'group_vcf.sh', 'filter.sh'],
            'bwa_mutect': ['pair_merge.sh', 'mutect.sh']},
 'RNASeq': {'All Workflows': ['make_group.sh'],
            'kallisto': ['k_align.sh',
                         'RNA-seq_limma.R',

#### Check which step calls a script

In [9]:
#should be the name of a shell script on the cluster (include sh extension)
target_script = "haplo.sh"

print(AddonsManager.get_steps_calling_script(ssh_client, scripts, target_script))

Executing cat /shared/workspace/Pipelines/config/tools.yaml
haplo.sh called from:
haplotype in the DNASeq bwa_gatk workflow



#### Check configuation entries for a step

In [11]:
step_name = "merge"

print(AddonsManager.get_step_config(ssh_client, scripts, step_name))

Executing cat /shared/workspace/Pipelines/config/tools.yaml
Executing python /shared/workspace/Pipelines/util/GetAllSpecificConfs.py
Executing cat /shared/workspace/Pipelines/scripts/DNASeq/merge.sh

tools.yaml configuration entry for merge step:
can_be_zipped: false
download_suffix: .g.vcf
input_is_output: true
script_path: DNASeq/merge
uses_chromosomes: false


DNASeq_bwa_gatk.yaml configuration entry for merge step:
merge:
- 4
- 'True'
Argument 1 is num_threads, Argument 2 is do_vcf_merging




#### Print out a script

In [14]:
#all targets must be set to a valid pipeline/workflow/script
#use cells above to check valid options

target_pipeline = "DNASeq"
target_workflow = "bwa_gatk"
target_script = "trim.sh"

loc, file_cat = AddonsManager.cat_script(ssh_client, scripts, target_pipeline, target_workflow, target_script)

print("".join(["#"]*len(loc)) + "\n{}\n".format(loc) + "".join(["#"]*len(loc)))
AddonsManager.show_script(file_cat)

Executing cat /shared/workspace/Pipelines/scripts/trim.sh
#############
All Pipelines
#############


### Check Software on Cluster

This section is used to check what software is installed on the cluster.

#### Get all software installed on cluster

In [10]:
software = AddonsManager.get_software_dict(ssh_client)
print("All Installed Software: ")
pprint(software)

Executing python /shared/workspace/Pipelines/util/GetSoftware.py
All Installed Software: 
{'FastQC': ['0.11.3'],
 'HTSeq': ['0.9.1 (installed with pip)'],
 'MultiQC': ['1.3 (installed with conda)'],
 'RSEM': ['1.3.0'],
 'STAR': ['2.3.0e', '2.5.1a', '2.5.3a'],
 'Trimmomatic': ['0.36'],
 'bedtools2': ['2.19.1'],
 'blat': ['36x1'],
 'bowtie': ['1.0.1'],
 'bowtie2': ['2.3.2-legacy'],
 'bwa': ['0.7.12-r1039'],
 'cutadapt': ['1.14 (installed with conda)'],
 'gatk': ['3.8-0'],
 'ghostscript': ['9.19'],
 'homer': ['4.8.3'],
 'java': ['jre1.8.0_144'],
 'kallisto': ['0.43.1'],
 'picard': ['1.96'],
 'sambamba': ['0.4.7'],
 'samblaster': ['0.1.21'],
 'samtools': ['1.1'],
 'tabix': ['0.2.6'],
 'vcftools': ['0.1.12b'],
 'weblogo': ['2.8']}


#### Check the version of a specific tool

In [16]:
#ignores case
target_tool = "cutadapt"

print(AddonsManager.check_tool_is_installed(software, target_tool))

cutadapt installed with version(s):
1.14 (installed with conda)
